In [4]:
import requests
#from api_keys import weather_api_key
import pandas as pd
import numpy as np
import datetime as dt
from splinter import Browser
from bs4 import BeautifulSoup
pd.set_option('max_colwidth', 400)

#### weather data : cloud forecasting
- Low cloud cover corresponds approximately to levels between 0 and 1800m above ground level.
- Medium cloud cover corresponds approximately to levels between 1800m and 6300m above ground level.
- High cloud cover corresponds approximately to levels higher than 6300m above ground level.
- Total cloud cover is the fraction of the sky covered by cloud at any height. Due to overlapping clouds at different heights, it may not correspond to the sum of the low, medium and high cloud covers.
- Effective cloud cover is a weighted sum of the low, medium and high level cloud covers relevant for solar energy forecasts. The low and medium level clouds have a higher weight than the high level clouds as the low and medium level clouds block more radiation.

In [2]:
base_url = 'https://api.meteomatics.com'

# Cloud forecast for 10 days from October 13, 2023
# Available levels: low, medium, high, effective, total
# Available measures: mean
# Available intervals: 1h, 2h, 6h, 12h, 24h
# Available units: octas, p

validdatetime = '2023-10-13T00:00:00ZP10D:PT1H'
location = '39.952583,-75.165222'  #Philadelphia as defalut 
format='json'
optionals = 'source=mix'
username = 'upenn_whang_sungim'
password = 'OD4p6y7qO5'

cloud_levels = ["low", "medium", "high", "effective", "total"]
all_cloud_data = {}

for level in cloud_levels :
    # 
    parameters = f'{level}_cloud_cover_mean_6h:p'

    cloud_url = f'{base_url}/{validdatetime}/{parameters}/{location}/{format}?{optionals}'
    print(cloud_url)
    response = requests.get(cloud_url, auth=(username, password))
    cloud_data = response.json()
    all_cloud_data[level] = cloud_data

https://api.meteomatics.com/2023-10-13T00:00:00ZP10D:PT1H/low_cloud_cover_mean_24h:p/39.952583,-75.165222/json?source=mix
https://api.meteomatics.com/2023-10-13T00:00:00ZP10D:PT1H/medium_cloud_cover_mean_24h:p/39.952583,-75.165222/json?source=mix
https://api.meteomatics.com/2023-10-13T00:00:00ZP10D:PT1H/high_cloud_cover_mean_24h:p/39.952583,-75.165222/json?source=mix
https://api.meteomatics.com/2023-10-13T00:00:00ZP10D:PT1H/effective_cloud_cover_mean_24h:p/39.952583,-75.165222/json?source=mix
https://api.meteomatics.com/2023-10-13T00:00:00ZP10D:PT1H/total_cloud_cover_mean_24h:p/39.952583,-75.165222/json?source=mix


In [5]:
# Function to convert to dataframes
def process_cloud_data(cloud_data, level):
    cloud_df = pd.DataFrame(cloud_data)
    cloud_df['date'] = pd.to_datetime(cloud_df['date'])
    cloud_df.set_index('date', inplace=True)
    cloud_df.index = cloud_df.index.strftime('%Y-%m-%d %H:%M')
    cloud_df.rename(columns={'value': f'{level} cloud cover mean'}, inplace=True)
    return cloud_df

In [6]:
cloud_data_frames = []

for i, level in enumerate(cloud_levels):
    cloud_data_frames.append(process_cloud_data(all_cloud_data[level]['data'][0]['coordinates'][0]['dates'], level))

# Merge the DataFrames
combined_df = cloud_data_frames[0]
for i in range(1, len(cloud_data_frames)):
    combined_df = pd.merge(combined_df, cloud_data_frames[i], on='date')

# Display DataFrame
combined_df.head(10)

,low cloud cover mean,medium cloud cover mean,high cloud cover mean,effective cloud cover mean,total cloud cover mean
date,,,,,
2023-10-13 00:00,0.0,13.6,24.5,25.8,38.0
2023-10-13 01:00,0.0,13.8,20.5,24.1,34.3
2023-10-13 02:00,0.0,14.0,16.5,22.3,30.5
2023-10-13 03:00,0.0,14.2,12.7,20.5,26.9
2023-10-13 04:00,0.0,14.3,8.7,18.6,23.0
2023-10-13 05:00,0.0,14.4,4.9,16.8,19.2
2023-10-13 06:00,0.0,14.6,1.6,15.4,16.2
2023-10-13 07:00,0.0,15.0,0.6,15.2,15.5
2023-10-13 08:00,0.0,17.7,0.2,17.8,17.9
